<a href="https://colab.research.google.com/github/Srivallimanognya/human_vs_AI-NLP-/blob/main/human_vs_AI_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers datasets pandas scikit-learn torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: datasets
    F

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

In [ ]:
# --- GPU Check Block ---
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))



CUDA available: True
GPU name: Tesla T4


In [ ]:
# --- Load Dataset Block ---
df = pd.read_csv("complete_dataset.csv")

# Your dataset already has numeric labels: 0 = Human, 1 = AI
# Just make sure they are integers
df["labels"] = df["label"].astype(int)

print("Unique numeric labels:", df["labels"].unique())
print("Dataset size:", len(df))

Unique numeric labels: [0 1]
Dataset size: 105000


In [ ]:

# --- Split Dataset Block ---
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["text"].astype(str).tolist(),
    df["labels"].tolist(),
    test_size=0.30,
    random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.50,
    random_state=42
)

print(f"Train size: {len(train_texts)}")
print(f"Validation size: {len(val_texts)}")
print(f"Test size: {len(test_texts)}")


Train size: 73500
Validation size: 15750
Test size: 15750


In [ ]:
#--- Convert to Hugging Face Dataset Block ---
train_df = pd.DataFrame({"text": train_texts, "labels": train_labels})
val_df = pd.DataFrame({"text": val_texts, "labels": val_labels})
test_df = pd.DataFrame({"text": test_texts, "labels": test_labels})

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)



In [ ]:
# --- Tokenization Block ---
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Format datasets for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/73500 [00:00<?, ? examples/s]

Map:   0%|          | 0/15750 [00:00<?, ? examples/s]

Map:   0%|          | 0/15750 [00:00<?, ? examples/s]

In [ ]:
# --- Model Block ---
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
# --- Training Setup Block ---
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,   # quick run; increase to 3+ for full training
    weight_decay=0.01,
    logging_steps=100,
    report_to="none"
)

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()



Step,Training Loss
100,0.227112
200,0.154802
300,0.179717
400,0.149326
500,0.212001
600,0.135469
700,0.143147
800,0.195093
900,0.140955
1000,0.248909


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=4594, training_loss=0.1843181962182224, metrics={'train_runtime': 1737.203, 'train_samples_per_second': 42.309, 'train_steps_per_second': 2.644, 'total_flos': 4901028815474688.0, 'train_loss': 0.1843181962182224, 'epoch': 1.0})

In [ ]:
# --- Evaluate Block ---
print("Validation Results:", trainer.evaluate(val_dataset))
print("Test Results:", trainer.evaluate(test_dataset))



Validation Results: {'eval_loss': 0.1863369196653366, 'eval_accuracy': 0.9352380952380952, 'eval_f1': 0.9372153145389635, 'eval_precision': 0.91304869273207, 'eval_recall': 0.9626960040465351, 'eval_runtime': 122.4184, 'eval_samples_per_second': 128.657, 'eval_steps_per_second': 8.046, 'epoch': 1.0}
Test Results: {'eval_loss': 0.1848488599061966, 'eval_accuracy': 0.9372063492063492, 'eval_f1': 0.9391496954408417, 'eval_precision': 0.9173076923076923, 'eval_recall': 0.9620572292953485, 'eval_runtime': 121.4409, 'eval_samples_per_second': 129.693, 'eval_steps_per_second': 8.111, 'epoch': 1.0}


In [ ]:
# --- Prediction Block ---
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "This is human generated text" if prediction == 0 else "This is AI generated text"

# Sample Prediction
sample_text = "Would you like me to help"
print("Sample Prediction:", classify_text(sample_text))

# User Input Prediction
user_text = input("Enter a sentence to classify: ")
print("Input:", user_text)
print("Model Prediction:", classify_text(user_text))


Sample Prediction: This is human generated text
Enter a sentence to classify: I was sitting on my porch yesterday evening, watching the sunset, and I couldn’t help but think about how peaceful life feels in those quiet moments."
Input: I was sitting on my porch yesterday evening, watching the sunset, and I couldn’t help but think about how peaceful life feels in those quiet moments."
Model Prediction: This is AI generated text
